# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing Data set!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing Data again. This time, however, we did things slightly different:
- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [2]:
age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [3]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]"
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0


## Run an linear model in Statsmodels

In [4]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [5]:
boston_features['MEDV'] = boston.target

In [23]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,RAD_0,RAD_1,TAX_0,TAX_1,TAX_2,MEDV
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0,24.0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0,21.6
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0,34.7
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0,33.4
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0,36.2


In [24]:
outcome = 'MEDV'
predictors = boston_features.columns
f = '+'.join(predictors)
formula = outcome + '~' + f
stats_model = ols(formula=formula, data=boston_features).fit()

In [25]:
stats_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.968e+31
Date:                Tue, 16 Jul 2019   Prob (F-statistic):               0.00
Time:                        20:59:04   Log-Likelihood:                 15471.
No. Observations:                 506   AIC:                        -3.091e+04
Df Residuals:                     492   BIC:                        -3.085e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.243e-14   5.32e-15     -2.339      0.020   -2.29e-14   -1.99e-15
CRIM        7.105e-15   6.25e-15      1.136      0.256   -5.18e-15    1.94e-14
INDUS       8.882e-16   1.08e-15      0.826      0.409   -1.23e-15       3e-15
CHAS       -3.331e-15   2.38e-15     -1.401      0.162      -8e-15    1.34e-15
RM          4.718e-15   1.26e-15      3.759      0.000    2.25e-15    7.18e-15
AGE        -1.887e-15   1.04e-15     -1.817      0.070   -3.93e-15    1.53e-16
DIS        -3.553e-15   5.64e-15     -0.629      0.529   -1.46e-14    7.54e-15
PTRATIO     8.049e-16    7.4e-16      1.088      0.277   -6.48e-16    2.26e-15
B          -1.221e-15   2.96e-15     -0.413      0.680   -7.03e-15    4.59e-15
LSTAT       1.138e-15   1.29e-15      0.885      0.376   -1.39e-15    3.66e-15
RAD_0      -5.995e-15   2.43e-15     -2.463      0.014   -1.08e-14   -1.21e-15
RAD_1      -5.773e-15   3.19e-15     -1.808      0.071    -1.2e-14    5.01e-16
TAX_0      -4.385e-15   2.14e-15     -2.047      0.041   -8.59e-15   -1.77e-16
TAX_1      -3.664e-15   2.09e-15     -1.756      0.080   -7.76e-15    4.35e-16
TAX_2      -4.885e-15   2.02e-15     -2.416      0.016   -8.86e-15   -9.13e-16
MEDV           1.0000   1.33e-16   7.52e+15      0.000       1.000       1.000
==============================================================================
Omnibus:                       82.703   Durbin-Watson:                   0.166
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              140.111
Skew:                          -0.986   Prob(JB):                     3.76e-31
Kurtosis:                       4.660   Cond. No.                     2.01e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.91e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [28]:
boston_features_constant = sm.add_constant(boston_features)
stats_model2 =  sm.OLS(boston.target, boston_features_constant).fit()
stats_model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.941e+31
Date:                Tue, 16 Jul 2019   Prob (F-statistic):               0.00
Time:                        21:02:33   Log-Likelihood:                 15467.
No. Observations:                 506   AIC:                        -3.091e+04
Df Residuals:                     492   BIC:                        -3.085e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.243e-14   5.35e-15     -2.323      0.021    -2.3e-14   -1.92e-15
CRIM        7.105e-15    6.3e-15      1.129      0.260   -5.26e-15    1.95e-14
INDUS       8.882e-16   1.08e-15      0.820      0.413   -1.24e-15    3.02e-15
CHAS       -3.331e-15   2.39e-15     -1.392      0.165   -8.03e-15    1.37e-15
RM          4.718e-15   1.26e-15      3.733      0.000    2.24e-15     7.2e-15
AGE        -1.887e-15   1.05e-15     -1.805      0.072   -3.94e-15    1.68e-16
DIS        -3.553e-15   5.68e-15     -0.625      0.532   -1.47e-14    7.61e-15
PTRATIO     8.049e-16   7.45e-16      1.081      0.280   -6.58e-16    2.27e-15
B          -1.221e-15   2.98e-15     -0.410      0.682   -7.07e-15    4.63e-15
LSTAT       1.138e-15   1.29e-15      0.879      0.380   -1.41e-15    3.68e-15
RAD_0      -5.995e-15   2.45e-15     -2.446      0.015   -1.08e-14   -1.18e-15
RAD_1      -5.773e-15   3.22e-15     -1.795      0.073   -1.21e-14    5.44e-16
TAX_0      -4.385e-15   2.16e-15     -2.033      0.043   -8.62e-15   -1.48e-16
TAX_1      -3.664e-15    2.1e-15     -1.744      0.082   -7.79e-15    4.63e-16
TAX_2      -4.885e-15   2.04e-15     -2.399      0.017   -8.88e-15   -8.85e-16
MEDV           1.0000   1.34e-16   7.47e+15      0.000       1.000       1.000
==============================================================================
Omnibus:                       75.198   Durbin-Watson:                   0.114
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              146.708
Skew:                          -0.846   Prob(JB):                     1.39e-32
Kurtosis:                       5.025   Cond. No.                     2.01e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.91e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Run the same model in Scikit-learn

In [32]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
scikit_model = linreg.fit(boston_features, boston_features['MEDV'])

In [33]:
scikit_model.intercept_

-3.552713678800501e-15

In [34]:
scikit_model.coef_

array([ 2.87137407e-15,  2.22295919e-15, -4.95903487e-15, -2.14867577e-15,
        9.51114330e-17,  7.15399269e-15, -1.11605471e-16, -2.81607561e-15,
        1.20797223e-15,  4.97889559e-16, -5.04828453e-16,  1.14448395e-16,
       -1.79024486e-16,  6.27854721e-17,  1.00000000e+00])

In [35]:
stats_model.params

Intercept   -1.243450e-14
CRIM         7.105427e-15
INDUS        8.881784e-16
CHAS        -3.330669e-15
RM           4.718448e-15
AGE         -1.887379e-15
DIS         -3.552714e-15
PTRATIO      8.049117e-16
B           -1.221245e-15
LSTAT        1.137979e-15
RAD_0       -5.995204e-15
RAD_1       -5.773160e-15
TAX_0       -4.385381e-15
TAX_1       -3.663736e-15
TAX_2       -4.884981e-15
MEDV         1.000000e+00
dtype: float64

## Remove the necessary variables to make sure the coefficients are the same for Scikit-learn vs Statsmodels

In [38]:
boston_features.drop('TAX_1', axis=1, inplace=True)

### Statsmodels

In [39]:
outcome = 'MEDV'
predictors = boston_features.columns
f = '+'.join(predictors)
formula = outcome + '~' + f
stats_model = ols(formula=formula, data=boston_features).fit()
stats_model.params

Intercept   -1.065814e-14
CRIM         1.509903e-14
INDUS        1.998401e-15
CHAS        -1.998401e-15
RM           1.443290e-15
AGE          8.881784e-16
DIS          2.664535e-15
PTRATIO     -4.996004e-16
B           -1.332268e-15
LSTAT       -1.582068e-15
RAD_0       -5.551115e-15
RAD_1       -6.883383e-15
TAX_0        1.054712e-15
TAX_2       -1.776357e-15
MEDV         1.000000e+00
dtype: float64

### Scikit-learn

In [40]:
linreg = LinearRegression()
scikit_model = linreg.fit(boston_features, boston_features['MEDV'])
print(scikit_model.intercept_)
print(scikit_model.coef_)

1.7763568394002505e-14
[-1.76115074e-14  3.31473577e-15 -1.48570731e-15 -1.61094393e-15
  4.36370224e-16  3.88011781e-15 -1.33449918e-15  4.30480898e-15
 -2.19597439e-15 -1.81204825e-16  1.67327037e-16  1.04211187e-15
  8.84005400e-16  1.00000000e+00]


## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

In [ ]:
x = {'CRIM': 0.15, 'INDUS': 6.07, 'CHAS': 1, 'RM':  6.1, 'AGE': 33.2, 'DIS': 7.6, 'PTRATIO': 17
 ,'B': 383, 'LSTAT': 10.87, 'RAD': 8, 'TAX': 284}
# (go through and change all values, but you need also min/max of o
#other .. etc. x_transform = [y['CRIM'] ]
#linreg.predict(y_predict)

## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing Data.